In [4]:
from pyswip.prolog import Prolog
from pyswip import registerForeign, Functor, call, Variable
import tempfile
import os

## Knowledge Base

- we can alternatively use prolog.assertz for dynamically constructing statements

In [5]:
KB = """

% Used for predicates that are added later
:- dynamic known/3, multivalued/1.

% Rules

% Rule 1:** if engine is not turning_over and battery is bad then problem is battery.

problem(battery):- \+engine(turning_over), battery(bad).

% Rule 2

problem(engine_oil_low) :- \+engine(turning_over), warning_light(oil).

% Rule 3

battery(bad) :- lights(weak).

% Rule 4

battery(bad) :- radio(weak).

% Rule 5

problem(engine_flooded):- engine(turning_over), smell(gas).

% Rule 6 if engine is turning_over and gas_gauge is empty then problem is out_of_gas.

problem(out_of_gas):- engine(turning_over), gas_gauge(empty).


% Askables
multivalued(engine).

engine(X):- ask(engine, X).

warning_light(X):- ask(warning_light, X).

lights(X):- ask(lights, X).

smell(X):- ask(smell, X).

radio(X):- ask(radio, X).

gas_gauge(X):- ask(gas_gauge, X).



% System Framework

ask(A, V):-
known(yes, A, V), % succeed if true
!.	% stop looking

ask(A, V):-
known(_, A, V), % fail if false
!, fail.

% If not multivalued, and already known to be something else, don't ask again for a different value.
ask(A, V):-
\+multivalued(A),
known(yes, A, V2),
V \== V2,
!, fail.

ask(A, V):-
read_py(A,V,Y), % get the answer
assertz(known(Y, A, V)), % remember it
Y == yes.	% succeed or fail

"""

## Interface Scaffold

In [6]:
prolog = Prolog()

# pull out Prolog built-ins into the Python name-space
retractall = Functor("retractall")
known = Functor("known", 3)

# user input and print functions
def write_py(X):
    print(str(X))
    sys.stdout.flush()
    return True


def read_py(A,V,Y):
    if isinstance(Y, Variable):
        response = input(str(A) + " is " + str(V) + "? ")
        Y.unify(response)
        return True
    else:
        return False

write_py.arity = 1
read_py.arity = 3

registerForeign(read_py)
registerForeign(write_py)

# Create a temporary file with the KB in it
(FD, name) = tempfile.mkstemp(suffix='.pl', text = "True")
with os.fdopen(FD, "w") as text_file:
    text_file.write(KB)
prolog.consult(name) # open the KB for consulting
os.unlink(name) # Remove the temporary file

call(retractall(known))
problem = [s for s in prolog.query("problem(X).", maxresult=1)]
print("Your problem is " + (problem[0]['X'] + "." if problem else "unknown."))


Your problem is unknown.
